# 1. Introduction

This notebook is one of the Formula 1 Marketing Campaign project pieces. The goal of this project is to build a Power BI dashboard where we can analyze the impact of F1 on social networks.

Succinctly, we will need two data packages. The first data package is about the championship (constructors, drivers, circuits, races, for example). These data were downloaded by connecting the [Ergast Developer API]( http://ergast.com/mrd/) with Power Query on a Microsoft Excel spreadsheet. The second data package is the information about social networks.

We will start the second package with the [Google Trends](https://trends.google.com/) information. This notebook was written to download this second data package and prepare them to use on Power BI.

It is important to say that Formula 1 is a private mark license. This project has only educational value and do not have any comercial purposes.

# 2. Data Loading

In [32]:
import pytrends
import pandas as pd

In [33]:
f1_raw = pd.read_excel("f1_from_api_results.xlsx")

display(f1_raw.head(3).T)

,0,1,2
RaceName,British Grand Prix,British Grand Prix,British Grand Prix
CircuitName,Silverstone Circuit,Silverstone Circuit,Silverstone Circuit
Locality,Silverstone,Silverstone,Silverstone
Country,UK,UK,UK
Attribute:lat,52.0786,52.0786,52.0786
Attribute:long,-1.01694,-1.01694,-1.01694
Attribute:circuitId,silverstone,silverstone,silverstone
Attribute:url.2,http://en.wikipedia.org/wiki/Silverstone_Circuit,http://en.wikipedia.org/wiki/Silverstone_Circuit,http://en.wikipedia.org/wiki/Silverstone_Circuit
Date,1950-05-13,1950-05-13,1950-05-13
Time,NaN,NaN,NaN


# 3. Data Preparation

We will collect data about interest on Google searches considering:

- Terms: `Formula 1`, and `F1`;

- Race (column `RaceName`);

- Circuit (column `CircuitName`);

- Driver (columns `GivenName` and `FamilyName`); and,

- Constructor (column `Name`)

We will focus on the 2021 and 2022 seasons (column `Attribute:season.1`).

In [34]:
cols_select = ["RaceName", "CircuitName", "GivenName", "FamilyName", "Name"]

f1_filter = f1_raw.loc[
    f1_raw["Attribute:season.1"].isin([2021, 2022]),
    cols_select
    ]

display(f1_filter.head(3).T)

,24960,24961,24962
RaceName,Bahrain Grand Prix,Bahrain Grand Prix,Bahrain Grand Prix
CircuitName,Bahrain International Circuit,Bahrain International Circuit,Bahrain International Circuit
GivenName,Lewis,Max,Valtteri
FamilyName,Hamilton,Verstappen,Bottas
Name,Mercedes,Red Bull,Mercedes


In [35]:
f1_filter.columns = ["RaceName", "CircuitName", "GivenName", "FamilyName", "ConstructorName"]

f1_filter["DriverName"] = f1_filter["GivenName"] + " " + f1_filter["FamilyName"]

f1_filter.drop(["GivenName", "FamilyName"], axis=1, inplace=True)

display(f1_filter.head(3).T)

,24960,24961,24962
RaceName,Bahrain Grand Prix,Bahrain Grand Prix,Bahrain Grand Prix
CircuitName,Bahrain International Circuit,Bahrain International Circuit,Bahrain International Circuit
ConstructorName,Mercedes,Red Bull,Mercedes
DriverName,Lewis Hamilton,Max Verstappen,Valtteri Bottas


In [79]:
# Generating list and dicts with the terms to search
kw_list = []
kw_dict = {}
for col, values in f1_filter.iteritems():
    kw_temp = [values.unique().tolist()]
    kw_list += kw_temp
    kw_dict = {**kw_dict, **{l:col for l in kw_temp[0]}}

RaceName, CircuitName, ConstructorName, DriverName = kw_list

# Filling the general terms
GeneralTerms = ["Formula 1", "F1"]

kw_list = [GeneralTerms] + kw_list
kw_dict = {**kw_dict, **{l:"GeneralTerms" for l in GeneralTerms}}

print("Terms to search:", *kw_list, sep="\n")

Terms to search:
['Formula 1', 'F1']
['Bahrain Grand Prix', 'Emilia Romagna Grand Prix', 'Portuguese Grand Prix', 'Spanish Grand Prix', 'Monaco Grand Prix', 'Azerbaijan Grand Prix', 'French Grand Prix', 'Styrian Grand Prix', 'Austrian Grand Prix', 'British Grand Prix', 'Hungarian Grand Prix', 'Belgian Grand Prix', 'Dutch Grand Prix', 'Italian Grand Prix', 'Russian Grand Prix', 'Turkish Grand Prix', 'United States Grand Prix', 'Mexico City Grand Prix', 'São Paulo Grand Prix', 'Qatar Grand Prix', 'Saudi Arabian Grand Prix', 'Abu Dhabi Grand Prix', 'Australian Grand Prix', 'Miami Grand Prix', 'Canadian Grand Prix', 'Singapore Grand Prix', 'Japanese Grand Prix']
['Bahrain International Circuit', 'Autodromo Enzo e Dino Ferrari', 'Autódromo Internacional do Algarve', 'Circuit de Barcelona-Catalunya', 'Circuit de Monaco', 'Baku City Circuit', 'Circuit Paul Ricard', 'Red Bull Ring', 'Silverstone Circuit', 'Hungaroring', 'Circuit de Spa-Francorchamps', 'Circuit Park Zandvoort', 'Autodromo Nazio

In [37]:
kw_list_all = [el for l in kw_list for el in l]

['Formula 1', 'F1', 'Bahrain Grand Prix', 'Emilia Romagna Grand Prix', 'Portuguese Grand Prix', 'Spanish Grand Prix', 'Monaco Grand Prix', 'Azerbaijan Grand Prix', 'French Grand Prix', 'Styrian Grand Prix', 'Austrian Grand Prix', 'British Grand Prix', 'Hungarian Grand Prix', 'Belgian Grand Prix', 'Dutch Grand Prix', 'Italian Grand Prix', 'Russian Grand Prix', 'Turkish Grand Prix', 'United States Grand Prix', 'Mexico City Grand Prix', 'São Paulo Grand Prix', 'Qatar Grand Prix', 'Saudi Arabian Grand Prix', 'Abu Dhabi Grand Prix', 'Australian Grand Prix', 'Miami Grand Prix', 'Canadian Grand Prix', 'Singapore Grand Prix', 'Japanese Grand Prix', 'Bahrain International Circuit', 'Autodromo Enzo e Dino Ferrari', 'Autódromo Internacional do Algarve', 'Circuit de Barcelona-Catalunya', 'Circuit de Monaco', 'Baku City Circuit', 'Circuit Paul Ricard', 'Red Bull Ring', 'Silverstone Circuit', 'Hungaroring', 'Circuit de Spa-Francorchamps', 'Circuit Park Zandvoort', 'Autodromo Nazionale di Monza', 'So

# 4. Data Collection

We created some lists with terms that we will look in Google Trends. The `pytrends` library can make many types of searches. We will use the Historical Hourly Interest (*returns historical, indexed, hourly data for when the keyword was searched most as shown on Google Trends' Interest Over Time section*).

It is important to say that we can search for more than 5 terms for time. In this way, we will need to split our requests. We will need to do this task two times. The first time worldwide and the focus in Brazil. A more accurate algorithm should analyze the impact in each country when the racers were happening. However, we will just use world and Brazil as parameters to our report.

In [39]:
from pytrends.request import TrendReq

def get_trends(
    geo: str,
    list_term: list = kw_list_all
    ) -> pd.DataFrame:
    """This function will use pytrends to 
    download data from Google Trends based
    on the location (Worldwide as default)
    and the a list of terms.
    """
    pytrends = TrendReq(hl='en-US', tz=180)

    trends_list = []
    for i in range(0, len(kw_list_all), 5):
        trends_part = pytrends.get_historical_interest(
            kw_list_all[i:(i+5)],
            year_start=2021,
            month_start=1,
            day_start=1,
            hour_start=0,
            year_end=2022,
            month_end=10,
            day_end=1,
            hour_end=0,
            cat=0,
            geo='',
            gprop='',
            sleep=60,
            frequency = 'daily').iloc[:,0:-1]

        trends_list.append(trends_part)

    df = pd.concat(trends_list, axis=1)

    df["geo"] = "WW" if geo == "" else geo

    return df

In [63]:
try:
    f1_ww = pd.read_csv("f1_trends_ww.csv")
except:
    f1_ww = get_trends(geo="")
    f1_ww.to_csv("f1_trends_ww.csv")
    f1_ww.reset_index(inplace=True)

In [41]:
try:
    f1_br = pd.read_csv("f1_trends_br.csv")
except:
    f1_br = get_trends(geo="BR")
    f1_br.to_csv("f1_trends_br.csv")
    f1_br.reset_index(inplace=True)

c:\Users\grego\AppData\Local\Programs\Python\Python310\lib\site-packages\pytrends\request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
c:\Users\grego\AppData\Local\Programs\Python\Python310\lib\site-packages\pytrends\request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
c:\Users\grego\AppData\Local\Programs\Python\Python310\lib\site-packages\pytrends\request.py:611: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
c:\Users\grego\AppData\Local\Programs\Python\Python310\lib\site-packages\pytrends\request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

# 6. Data Process

As you saw, we created two dataframes. We will now process them (transform format from long to wide and add a new column), merge them and export the final dataset.

In [80]:
def f1_process(df: pd.DataFrame,
    dict_replace: dict = kw_dict
    ) -> pd.DataFrame:
    """This function will melt a dataframe and 
    add a column with the related term base on
    the keyWord column.
    """
    df = df.melt(
        id_vars=["date", "geo"],
        var_name="keyWord",
        value_name="interestOverTime")

    df["keyGroup"] = df["keyWord"].replace(dict_replace)

    return df

In [83]:
f1_trends = pd.concat(
    [f1_process(f1_ww),
     f1_process(f1_br)],
    axis=0,
    ignore_index=True
)

f1_trends.to_csv("f1_trends.csv")

# 7. Conclusion

Using pytrends and some lines of code, we downloaded data from Google Trends. The data is now ready to be used wherever we want.